# 13b_encuestas Include latest polls (Pollsters Jesus, Ross, Meli in Providencia may 2025)

Some polls were created after the overall polls processing in order to strengthen the data. This notebook __loads the manually drawn latest polls and the Google Forms poll data__, and creates a new gpkg with the poll data and drawn route.

* __Output: final_routes.gpkg__

In [1]:
#Autopep8

## Import libraries

In [2]:
first_folder_path = '../../../'

In [3]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Load data (Google Forms polls)

In [4]:
surveys_file = 'Encuesta_caminata(Guadalajara)(1-553).csv'
surveys = pd.read_csv(first_folder_path + f"data/processed/volvo/encuestas/01_encuestas_google_forms/{surveys_file}")

# Show
print(surveys.shape)
surveys.head(2)

(553, 65)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [5]:
surveys.Encuestador.unique()

array(['DS', 'JM', 'DP', 'DP ', '049', 'MV', 'RV', 'EE1V002', 'EE1V004',
       'EE1V001', 'EE1V005', 'EE1V003', 'Jesus Corona', 'Jesús Corona',
       'rv', 'MeV'], dtype=object)

## Filter and fix codes

In [6]:
# Pollster a) - Jesus
encuestador_lst = ['Jesus Corona', 'Jesús Corona']
surveys_jesus = surveys.loc[surveys.Encuestador.isin(encuestador_lst)].copy()

# Rename codes
surveys_jesus["code"] = 0
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-001', "code"] = 'JC1V001'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-002', "code"] = 'JC1V002'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-003', "code"] = 'JC1V003'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-004', "code"] = 'JC1V004'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-005', "code"] = 'JC1V005'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-006', "code"] = 'JC1V006'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 M-007', "code"] = 'JC1V007'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 M-008', "code"] = 'JC1V008'
surveys_jesus

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
528,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001
529,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002
530,531,5/9/25 12:53:14,5/9/25 12:57:26,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-003,TA_Providencia,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,En desacuerdo,JC1V003
531,532,5/9/25 12:57:29,5/9/25 14:43:21,anonymous,NaN,NaN,Jesús Corona,5/24/2025,JC-1 V-004,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V004
532,533,5/9/25 14:43:23,5/9/25 14:50:40,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-1 V-005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V005
533,534,5/9/25 14:50:42,5/9/25 14:56:06,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-01 V-006,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo",JC1V006
534,535,5/9/25 14:56:21,5/9/25 15:08:22,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-01 M-007,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,JC1V007
535,536,5/9/25 15:08:24,5/9/25 15:20:28,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-1 M-008,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,JC1V008


In [8]:
# Pollster b) - Meli
encuestador_lst = ['MeV']
surveys_meli = surveys.loc[surveys.Encuestador.isin(encuestador_lst)].copy()

# Rename codes
surveys_meli["code"] = 0
surveys_meli.loc[surveys_meli["Encuesta #"]=='0001', "code"] = 'MeV1V001'
surveys_meli.loc[surveys_meli["Encuesta #"]=='002', "code"] = 'MeV1V002'
surveys_meli.loc[surveys_meli["Encuesta #"]=='003', "code"] = 'MeV1V003'
surveys_meli.loc[surveys_meli["Encuesta #"]=='004', "code"] = 'MeV1V004'
surveys_meli.loc[surveys_meli["Encuesta #"]=='005', "code"] = 'MeV1V005'
surveys_meli.loc[surveys_meli["Encuesta #"]=='006', "code"] = 'MeV1V006'
surveys_meli.loc[surveys_meli["Encuesta #"]=='007', "code"] = 'MeV1V007'
surveys_meli.loc[surveys_meli["Encuesta #"]=='008', "code"] = 'MeV1V008'
surveys_meli.loc[surveys_meli["Encuesta #"]=='009', "code"] = 'MeV1V009'
surveys_meli.loc[surveys_meli["Encuesta #"]=='010', "code"] = 'MeV1V010'
surveys_meli

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
543,544,5/11/25 19:50:28,5/11/25 19:59:26,anonymous,NaN,NaN,MeV,3/18/2025,0001,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V001
544,545,5/11/25 20:00:52,5/11/25 20:07:21,anonymous,NaN,NaN,MeV,3/18/2025,002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V002
545,546,5/11/25 20:07:30,5/11/25 20:15:38,anonymous,NaN,NaN,MeV,3/18/2025,003,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V003
546,547,5/11/25 20:15:47,5/11/25 20:17:53,anonymous,NaN,NaN,MeV,3/18/2025,004,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V004
547,548,5/11/25 20:18:06,5/11/25 20:29:49,anonymous,NaN,NaN,MeV,3/18/2025,005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,MeV1V005
548,549,5/11/25 20:29:58,5/11/25 20:32:13,anonymous,NaN,NaN,MeV,3/18/2025,006,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V006
549,550,5/11/25 20:32:22,5/11/25 20:36:13,anonymous,NaN,NaN,MeV,3/18/2025,007,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,MeV1V007
550,551,5/11/25 20:36:23,5/11/25 20:41:02,anonymous,NaN,NaN,MeV,3/18/2025,008,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,MeV1V008
551,552,5/11/25 20:41:04,5/11/25 20:45:47,anonymous,NaN,NaN,MeV,3/18/2025,009,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V009
552,553,5/11/25 20:45:49,5/11/25 20:49:13,anonymous,NaN,NaN,MeV,3/18/2025,010,TA_Providencia,...,De acuerdo,En desacuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,Totalmente en desacuerdo,MeV1V010


In [7]:
# Pollster b) - Ross
encuestador_lst = ['RV', 'rv']
surveys_ross = surveys.loc[(surveys.Encuestador.isin(encuestador_lst))&(surveys["Territorio de análisis"]=='TA_Providencia')].copy()

surveys_ross

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
510,511,5/7/25 23:11:14,5/7/25 23:16:03,anonymous,NaN,NaN,RV,3/18/2025,RV1V001,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
511,512,5/7/25 23:16:10,5/7/25 23:18:09,anonymous,NaN,NaN,RV,3/18/2025,RV1V002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
512,513,5/7/25 23:18:11,5/7/25 23:20:00,anonymous,NaN,NaN,RV,3/18/2025,RV1V003,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
513,514,5/7/25 23:20:05,5/7/25 23:26:07,anonymous,NaN,NaN,RV,3/18/2025,RV1V004,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo
514,515,5/7/25 23:26:15,5/7/25 23:30:25,anonymous,NaN,NaN,RV,3/18/2025,RV1V005,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
515,516,5/7/25 23:30:37,5/7/25 23:32:11,anonymous,NaN,NaN,RV,3/18/2025,RV1V006,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
516,517,5/7/25 23:32:20,5/7/25 23:36:39,anonymous,NaN,NaN,RV,3/18/2025,RVV1V007,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
517,518,5/7/25 23:36:41,5/7/25 23:42:13,anonymous,NaN,NaN,RV,3/18/2025,RV1V008,TA_Providencia,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo
518,519,5/7/25 23:42:16,5/7/25 23:46:28,anonymous,NaN,NaN,RV,3/18/2025,RV1V009,TA_Providencia,...,Totalmente en desacuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo
519,520,5/7/25 23:46:32,5/7/25 23:50:05,anonymous,NaN,NaN,RV,3/18/2025,RV1V010,TA_Providencia,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,En desacuerdo,De acuerdo


### __Problem:__ Found multiple entries for Rossana's polls. Seems duplicated.

In [25]:
surveys.columns

Index(['ID', 'Hora de inicio', 'Hora de finalización', 'Correo electrónico',
       'Nombre', 'Hora de la última modificación', 'Encuestador',
       'Fecha de realización', 'Encuesta #', 'Territorio de análisis',
       'Lugar donde se realizó la encuesta', 'Género', 'Edad (años)',
       'Estrato socioeconómico (NO APLICA)', 'Ocupación principal',
       'Nivel educativo completado',
       '¿Poseen alguno de los siguientes vehículos en su vivienda?',
       'Indique su lugar de residencia (Municipio/comuna/barrio)',
       '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
       '¿Cómo suple las necesidades de equipamientos educativos?',
       '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
       '¿Cuáles?',
       '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
       '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
       'En una semana típica, ¿Cuántas veces 

In [33]:
surveys_ross[['ID', 	
              'Hora de inicio',
              'Hora de finalización',
              'Correo electrónico',
              'Nombre',
              'Hora de la última modificación',
              'Encuestador',
              'Fecha de realización',
              'Encuesta #',
              'Territorio de análisis',
              'Lugar donde se realizó la encuesta', 
              'Género', 
              'Edad (años)',
              'Estrato socioeconómico (NO APLICA)', 
              'Ocupación principal',
              'Nivel educativo completado',
              '¿Poseen alguno de los siguientes vehículos en su vivienda?',
              'Indique su lugar de residencia (Municipio/comuna/barrio)',
              ]]

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,Lugar donde se realizó la encuesta,Género,Edad (años),Estrato socioeconómico (NO APLICA),Ocupación principal,Nivel educativo completado,¿Poseen alguno de los siguientes vehículos en su vivienda?,Indique su lugar de residencia (Municipio/comuna/barrio)
510,511,5/7/25 23:11:14,5/7/25 23:16:03,anonymous,NaN,NaN,RV,3/18/2025,RV1V001,TA_Providencia,Parque Rubén Darío,Femenino,50-59,NaN,Independiente,Pregrado,Auto - coche;,"Tlajomulco, Real del valle"
511,512,5/7/25 23:16:10,5/7/25 23:18:09,anonymous,NaN,NaN,RV,3/18/2025,RV1V002,TA_Providencia,Parque Rubén Darío,Masculino,20-29,NaN,Independiente,Técnico,Auto - coche;Motocicleta;Bicicleta;,"Zapopan, Loma Bonita"
512,513,5/7/25 23:18:11,5/7/25 23:20:00,anonymous,NaN,NaN,RV,3/18/2025,RV1V003,TA_Providencia,Parque Rubén Darío,Masculino,40-49,NaN,Independiente,Técnico,Auto - coche;Motocicleta;Bicicleta;,"Tlaquepaque, Buenos Aires"
513,514,5/7/25 23:20:05,5/7/25 23:26:07,anonymous,NaN,NaN,RV,3/18/2025,RV1V004,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Independiente,Pregrado,Auto - coche;Bicicleta;,"Zapopan, Puerta del Bosque"
514,515,5/7/25 23:26:15,5/7/25 23:30:25,anonymous,NaN,NaN,RV,3/18/2025,RV1V005,TA_Providencia,Parque Rubén Darío,Masculino,30-39,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"
515,516,5/7/25 23:30:37,5/7/25 23:32:11,anonymous,NaN,NaN,RV,3/18/2025,RV1V006,TA_Providencia,Parque Rubén Darío,Femenino,50-59,NaN,Empleado/a,Secundaria,Ninguno;,"Guadalajara, Hidalgo"
516,517,5/7/25 23:32:20,5/7/25 23:36:39,anonymous,NaN,NaN,RV,3/18/2025,RVV1V007,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Independiente,Técnico,Auto - coche;Bicicleta;,"Guadalajara, Providencia"
517,518,5/7/25 23:36:41,5/7/25 23:42:13,anonymous,NaN,NaN,RV,3/18/2025,RV1V008,TA_Providencia,Parque Rubén Darío,Masculino,10-19,NaN,Empleado/a,Técnico,Bicicleta;,"Zapopan, Arenales Tapatíos"
518,519,5/7/25 23:42:16,5/7/25 23:46:28,anonymous,NaN,NaN,RV,3/18/2025,RV1V009,TA_Providencia,Parque Rubén Darío,Masculino,30-39,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"
519,520,5/7/25 23:46:32,5/7/25 23:50:05,anonymous,NaN,NaN,RV,3/18/2025,RV1V010,TA_Providencia,Parque Rubén Darío,Masculino,40-49,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"


In [34]:
surveys_ross[['ID','Encuesta #',
              '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
              '¿Cómo suple las necesidades de equipamientos educativos?',
              '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
              '¿Cuáles?',
              '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
              '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?',
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)']]

,ID,Encuesta #,¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?,¿Cómo suple las necesidades de equipamientos educativos?,¿Cómo suple las necesidades de equipamientos de cultura o deportivos?,¿Cuáles?,¿Hacia qué tipo de equipamiento/sitio de interés se dirige?,¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado),"En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?","En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)"
510,511,RV1V001,Si,NaN,NaN,Parques;,Cultural/Deportivo,Carro particular - coche,1,NaN
511,512,RV1V002,Si,NaN,NaN,Parques;,Cultural/Deportivo,Motocicleta,5 o más,NaN
512,513,RV1V003,Si,NaN,NaN,Parques;,Cultural/Deportivo,Carro particular - coche,4,NaN
513,514,RV1V004,Si,NaN,NaN,Parques;,Cultural/Deportivo,Caminata,NaN,5 o más
514,515,RV1V005,Si,NaN,NaN,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más
515,516,RV1V006,Si,NaN,NaN,Comercial;Parques;,Cultural/Deportivo,Bus,3,NaN
516,517,RVV1V007,Si,NaN,NaN,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más
517,518,RV1V008,Si,NaN,NaN,Parques;,Cultural/Deportivo,Caminata,NaN,1
518,519,RV1V009,Si,NaN,NaN,Comercial;Parques;,Cultural/Deportivo,Caminata,NaN,5 o más
519,520,RV1V010,Si,NaN,NaN,Salud;Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más


In [35]:
surveys_ross[['Debido a la lejanía del equipamiento X',
       'Debido a la ausencia de andenes',
       'Porque no me siento seguro/a frente a otros vehículos',
       'Porque no me siento seguro/a frente a posibles delitos en mi trayecto',
       'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles',
       'Debido a que no me siento cómodo/a por el calor',
       'Debido a que siento que los andenes muy saturados de gente',
       'Debido a que el paisaje a lo largo del recorrido no es agradable',
       'La cercanía del equipamiento X', 'La presencia de andenes',
       'Sentirme seguro/a frente a otros vehículos',
       'Sentirme seguro/a frente a posibles delitos en mi trayecto',
       'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido',
       'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido',
       'Sentirme cómodo/a por la sombra generada por la vegetación',
       'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
       'Que el paisaje sea agradable',
       '¿Se encuentra usted en alguna condición de discapacidad?',
       'Indique cuál de las siguientes condiciones de discapacidad presenta']]

,Debido a la lejanía del equipamiento X,Debido a la ausencia de andenes,Porque no me siento seguro/a frente a otros vehículos,Porque no me siento seguro/a frente a posibles delitos en mi trayecto,Debido al esfuerzo que debo hacer a causa de la pendiente de las calles,Debido a que no me siento cómodo/a por el calor,Debido a que siento que los andenes muy saturados de gente,Debido a que el paisaje a lo largo del recorrido no es agradable,La cercanía del equipamiento X,La presencia de andenes,Sentirme seguro/a frente a otros vehículos,Sentirme seguro/a frente a posibles delitos en mi trayecto,La posibilidad de hacer múltiples vueltas/trámites en mi recorrido,La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido,Sentirme cómodo/a por la sombra generada por la vegetación,Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido,Que el paisaje sea agradable,¿Se encuentra usted en alguna condición de discapacidad?,Indique cuál de las siguientes condiciones de discapacidad presenta
510,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,En desacuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,No,NaN
514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Importante,Importante,Muy importante,No es importante,No es importante,Importante,Importante,Importante,No,NaN
515,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,En desacuerdo,Totalmente en desacuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Muy importante,Muy importante,Importante,Es poco importante,Importante,Moderadamente importante,Moderadamente importante,No,NaN
517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Importante,Muy importante,Muy importante,Muy importante,Muy importante,Es poco importante,Muy importante,No es importante,Muy importante,No,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Importante,Muy importante,Moderadamente importante,Importante,Moderadamente importante,No es importante,Es poco importante,No,NaN
519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Importante,Importante,Importante,Moderadamente importante,Importante,Muy importante,Muy importante,No,NaN


In [60]:
surveys_ross[['Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)',
              'Indique la hora de salida de su lugar de origen de recorrido (formato militar)',
              'Ingrese los minutos de salida',
              #'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana)',
              '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)',
              'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...',
              '¿Por qué razón utiliza la CAMINATA como medio de transporte?',
              '¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?'
             ]]

,"Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)",Indique la hora de salida de su lugar de origen de recorrido (formato militar),Ingrese los minutos de salida,¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos),Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...,¿Por qué razón utiliza la CAMINATA como medio de transporte?,¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?
510,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,Eulogio Parra y Rubén Darío,17.0,30.0,12.0,Inicia en Eulogio Parra entre Ontario y Rubén ...,Pasear mascota;,Carro particular - coche
514,Otawa esquina con Florencia,17.0,0.0,5.0,"Inicia en otawa esquina con Florencia, camina ...",Por cercanía al lugar de destino;,Carro particular - coche
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,Calle Florencia esquina con Monza,18.0,10.0,5.0,"Inicia en Monza con Florencia, camina sobre Fl...",Pasear mascota;,Carro particular - coche
517,Av. Terranova,18.0,35.0,20.0,"Inicia en Av Terranova y Pablo Neruda, camina ...",Distracción;,Bus
518,Terranova esquina con José María VIgil,18.0,40.0,5.0,Camina por Terranova y José María Vigil hacia ...,Pasear mascota;,Carro particular - coche
519,"José María Vigil, entre Terranova y Rubén Darío",18.0,15.0,5.0,Inicia sobre José María Vigil casi esquina con...,Pasear mascota;,Carro particular - coche


In [50]:
surveys_ross[['ID','Encuesta #','Column', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

,ID,Encuesta #,Column,2,3,4,5,6,7,8,9,10,11,12
510,511,RV1V001,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
511,512,RV1V002,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
512,513,RV1V003,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
513,514,RV1V004,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo
514,515,RV1V005,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
515,516,RV1V006,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
516,517,RVV1V007,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
517,518,RV1V008,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo
518,519,RV1V009,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente en desacuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo
519,520,RV1V010,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,En desacuerdo,De acuerdo


### __Approached solution:__ Consider only entries with date 5/7/25 (Where pollster reported polls were performed.)

In [9]:
date_ids = [511,512,513,514,515,516,517,518,519,520]
surveys_ross_f = surveys_ross.loc[surveys_ross.ID.isin(date_ids)].copy()

# Rename codes
surveys_ross_f["code"] = 0
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V001', "code"] = 'RV1V001'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V002', "code"] = 'RV1V002'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V003', "code"] = 'RV1V003'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V004', "code"] = 'RV1V004'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V005', "code"] = 'RV1V005'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V006', "code"] = 'RV1V006'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RVV1V007', "code"] = 'RV1V007'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V008', "code"] = 'RV1V008'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V009', "code"] = 'RV1V009'
surveys_ross_f.loc[surveys_ross_f["Encuesta #"]=='RV1V010', "code"] = 'RV1V010'
surveys_ross_f

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
510,511,5/7/25 23:11:14,5/7/25 23:16:03,anonymous,NaN,NaN,RV,3/18/2025,RV1V001,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",RV1V001
511,512,5/7/25 23:16:10,5/7/25 23:18:09,anonymous,NaN,NaN,RV,3/18/2025,RV1V002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",RV1V002
512,513,5/7/25 23:18:11,5/7/25 23:20:00,anonymous,NaN,NaN,RV,3/18/2025,RV1V003,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",RV1V003
513,514,5/7/25 23:20:05,5/7/25 23:26:07,anonymous,NaN,NaN,RV,3/18/2025,RV1V004,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,RV1V004
514,515,5/7/25 23:26:15,5/7/25 23:30:25,anonymous,NaN,NaN,RV,3/18/2025,RV1V005,TA_Providencia,...,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,RV1V005
515,516,5/7/25 23:30:37,5/7/25 23:32:11,anonymous,NaN,NaN,RV,3/18/2025,RV1V006,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",RV1V006
516,517,5/7/25 23:32:20,5/7/25 23:36:39,anonymous,NaN,NaN,RV,3/18/2025,RVV1V007,TA_Providencia,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,RV1V007
517,518,5/7/25 23:36:41,5/7/25 23:42:13,anonymous,NaN,NaN,RV,3/18/2025,RV1V008,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,RV1V008
518,519,5/7/25 23:42:16,5/7/25 23:46:28,anonymous,NaN,NaN,RV,3/18/2025,RV1V009,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,RV1V009
519,520,5/7/25 23:46:32,5/7/25 23:50:05,anonymous,NaN,NaN,RV,3/18/2025,RV1V010,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,En desacuerdo,De acuerdo,RV1V010


In [10]:
latest_surveys = pd.concat([surveys_jesus,surveys_meli])
latest_surveys = pd.concat([latest_surveys,surveys_ross_f])

# Show
print(latest_surveys.shape)
latest_surveys

(28, 66)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
528,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001
529,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002
530,531,5/9/25 12:53:14,5/9/25 12:57:26,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-003,TA_Providencia,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,En desacuerdo,JC1V003
531,532,5/9/25 12:57:29,5/9/25 14:43:21,anonymous,NaN,NaN,Jesús Corona,5/24/2025,JC-1 V-004,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V004
532,533,5/9/25 14:43:23,5/9/25 14:50:40,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-1 V-005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V005
533,534,5/9/25 14:50:42,5/9/25 14:56:06,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-01 V-006,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo",JC1V006
534,535,5/9/25 14:56:21,5/9/25 15:08:22,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-01 M-007,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,JC1V007
535,536,5/9/25 15:08:24,5/9/25 15:20:28,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-1 M-008,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,JC1V008
543,544,5/11/25 19:50:28,5/11/25 19:59:26,anonymous,NaN,NaN,MeV,3/18/2025,0001,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V001
544,545,5/11/25 20:00:52,5/11/25 20:07:21,anonymous,NaN,NaN,MeV,3/18/2025,002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V002


## Load latest drawn routes

In [11]:
JC1V001 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/JC1V001.shp")
JC1V002 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/JC1V002.shp")
JC1V004 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/JC1V004.shp")
JC1V005 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/JC1V005.shp")
JC1V008 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/JC1V008.shp")
#MEV1V002 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MEV1V002.shp") # No route
MeV1V003 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V003.shp")
MeV1V005 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V005.shp")
MeV1V006 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V006.shp")
MeV1V007= gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V007.shp")
MeV1V008 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V008.shp")
MeV1V009 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V009.shp")
MeV1V010 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/MeV1V010.shp")
RV1V005 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/RV1V005.shp")
RV1V007 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/RV1V007.shp")
RV1V008 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/RV1V008.shp")
RV1V009 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/RV1V009.shp")
RV1V010 = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_complementarias/RV1V010.shp")

# Show example
JC1V001

,id,geometry
0,1,"LINESTRING (668461.981 2288949.643, 668280.506..."


In [12]:
latest = {'JC1V001':JC1V001,
          'JC1V002':JC1V002,
          'JC1V004':JC1V004,
          'JC1V005':JC1V005,
          'JC1V008':JC1V008,
          'MeV1V003':MeV1V003,
          'MeV1V005':MeV1V005,
          'MeV1V006':MeV1V006,
          'MeV1V007':MeV1V007,
          'MeV1V008':MeV1V008,
          'MeV1V009':MeV1V009,
          'MeV1V010':MeV1V010,
          'RV1V005':RV1V005,
          'RV1V007':RV1V007,
          'RV1V008':RV1V008,
          'RV1V009':RV1V009,
          'RV1V010':RV1V010}

## Join all latest routes, assign codes and create gdf

In [13]:
# Each route's data is registered as a dict. in rows lst
rows = []
for code, gdf in latest.items():
    rows.append({'code':code,
                'geometry':gdf.geometry[0]})

# Rows dict is converted to DataFrame
latest_routes_df = pd.DataFrame(rows)

# Show
print(latest_routes_df.shape)
latest_routes_df

(17, 2)


,code,geometry
0,JC1V001,LINESTRING (668461.9805032494 2288949.64323436...
1,JC1V002,LINESTRING (668277.7749232971 2289017.68267455...
2,JC1V004,LINESTRING (668278.5582454879 2289010.05833857...
3,JC1V005,LINESTRING (668452.4557718078 2288907.91312491...
4,JC1V008,LINESTRING (667358.3113359347 2288494.11012234...
5,MeV1V003,LINESTRING (668231.4119118408 2288895.01289577...
6,MeV1V005,LINESTRING (668278.155087195 2289010.769555157...
7,MeV1V006,LINESTRING (668189.1962168223 2288623.24171131...
8,MeV1V007,LINESTRING (668289.5961523671 2289005.26316015...
9,MeV1V008,LINESTRING (668266.8363863566 2288945.91645846...


## Merge drawn routes and surveys

In [14]:
latest_surveys.head(2)

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
528,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001
529,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002


In [22]:
latest_routes_data = pd.merge(latest_surveys,latest_routes_df,on='code')
latest_routes_gdf = gpd.GeoDataFrame(latest_routes_data, geometry='geometry',crs='EPSG:32613')

# Show
print(latest_routes_gdf.crs)
print(latest_routes_gdf.shape)
latest_routes_gdf

EPSG:32613
(17, 67)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,5,6,7,8,9,10,11,12,code,geometry
0,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001,"LINESTRING (668461.981 2288949.643, 668280.506..."
1,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002,"LINESTRING (668277.775 2289017.683, 668281.117..."
2,532,5/9/25 12:57:29,5/9/25 14:43:21,anonymous,NaN,NaN,Jesús Corona,5/24/2025,JC-1 V-004,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V004,"LINESTRING (668278.558 2289010.058, 668268.271..."
3,533,5/9/25 14:43:23,5/9/25 14:50:40,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-1 V-005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V005,"LINESTRING (668452.456 2288907.913, 668444.205..."
4,536,5/9/25 15:08:24,5/9/25 15:20:28,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-1 M-008,TA_Providencia,...,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,JC1V008,"LINESTRING (667358.311 2288494.11, 667701.72 2..."
5,546,5/11/25 20:07:30,5/11/25 20:15:38,anonymous,NaN,NaN,MeV,3/18/2025,003,TA_Providencia,...,De acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V003,"LINESTRING (668231.412 2288895.013, 668237.041..."
6,548,5/11/25 20:18:06,5/11/25 20:29:49,anonymous,NaN,NaN,MeV,3/18/2025,005,TA_Providencia,...,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,MeV1V005,"LINESTRING (668278.155 2289010.77, 668283.05 2..."
7,549,5/11/25 20:29:58,5/11/25 20:32:13,anonymous,NaN,NaN,MeV,3/18/2025,006,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V006,"LINESTRING (668189.196 2288623.242, 668205.104..."
8,550,5/11/25 20:32:22,5/11/25 20:36:13,anonymous,NaN,NaN,MeV,3/18/2025,007,TA_Providencia,...,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,MeV1V007,"LINESTRING (668289.596 2289005.263, 668266.836..."
9,551,5/11/25 20:36:23,5/11/25 20:41:02,anonymous,NaN,NaN,MeV,3/18/2025,008,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,MeV1V008,"LINESTRING (668266.836 2288945.916, 668240.895..."


## Format the same wah preliminar_routes is formated

In [23]:
latest_routes_gdf.rename(columns={'ID': 'field_1',
                             'Hora de inicio': 'field_2',
                             'Hora de finalización':'field_3',
                             'Correo electrónico':'field_4',
                             'Nombre':'field_5',
                             'Hora de la última modificación':'field_6',
                             'Encuestador':'field_7',
                             'Fecha de realización':'field_8',
                             'Encuesta #':'field_9',
                             #field_10 es el Territorio de análisis en número
                             'Territorio de análisis':'field_11',
                             #field_12 no existe
                             'Lugar donde se realizó la encuesta':'field_13',
                             'Género':'field_14',
                             'Edad (años)':'field_15',
                             'Estrato socioeconómico (NO APLICA)':'field_16',
                             'Ocupación principal':'field_17',
                             'Nivel educativo completado':'field_18',
                             '¿Poseen alguno de los siguientes vehículos en su vivienda?':'field_19',
                             'Indique su lugar de residencia (Municipio/comuna/barrio)':'field_20',
                             '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?':'field_21',
                             '¿Cómo suple las necesidades de equipamientos educativos?':'field_22',
                             '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?':'field_23',
                             '¿Cuáles?':'field_24',
                             '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?':'field_25',
                             '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)': 'field_26',
                             'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?': 'field_27',
                             'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)': 'field_28',
                             'Debido a la lejanía del equipamiento X': 'field_29',
                             'Debido a la ausencia de andenes': 'field_30',
                             'Porque no me siento seguro/a frente a otros vehículos': 'field_31',
                             'Porque no me siento seguro/a frente a posibles delitos en mi trayecto': 'field_32',
                             'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles': 'field_33',
                             'Debido a que no me siento cómodo/a por el calor': 'field_34',
                             'Debido a que siento que los andenes muy saturados de gente': 'field_35',
                             'Debido a que el paisaje a lo largo del recorrido no es agradable': 'field_36',
                             'La cercanía del equipamiento X': 'field_37',
                             'La presencia de andenes': 'field_38',
                             'Sentirme seguro/a frente a otros vehículos': 'field_39',
                             'Sentirme seguro/a frente a posibles delitos en mi trayecto': 'field_40',
                             'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido': 'field_41',
                             'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido': 'field_42',
                             'Sentirme cómodo/a por la sombra generada por la vegetación': 'field_43',
                             'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido': 'field_44',
                             'Que el paisaje sea agradable': 'field_45',
                             '¿Se encuentra usted en alguna condición de discapacidad?': 'field_46',
                             'Indique cuál de las siguientes condiciones de discapacidad presenta': 'field_47',
                             'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)': 'field_48',
                             'Indique la hora de salida de su lugar de origen de recorrido (formato militar)': 'field_49',
                             'Ingrese los minutos de salida': 'field_50',
                             'Indique el lugar de destino de su recorrido lo más precios posible\xa0(Punto de referencia, intersección más cercana)': 'field_51', #Needed \xa0 for it to be read 
                             '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)': 'field_52',
                             'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...': 'field_53',
                             '¿Por qué razón utiliza la CAMINATA como medio de transporte?': 'field_54',
                             '¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?': 'field_55',
                             'Column': 'field_56',
                             '2': 'field_57',
                             '3': 'field_58',
                             '4': 'field_59',
                             '5': 'field_60',
                             '6': 'field_61',
                             '7': 'field_62',
                             '8': 'field_63',
                             '9': 'field_64',
                             '10': 'field_65',
                             '11': 'field_66',
                             '12': 'field_67'
                            },inplace=True)

latest_routes_gdf.columns

Index(['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6',
       'field_7', 'field_8', 'field_9', 'field_11', 'field_13', 'field_14',
       'field_15', 'field_16', 'field_17', 'field_18', 'field_19', 'field_20',
       'field_21', 'field_22', 'field_23', 'field_24', 'field_25', 'field_26',
       'field_27', 'field_28', 'field_29', 'field_30', 'field_31', 'field_32',
       'field_33', 'field_34', 'field_35', 'field_36', 'field_37', 'field_38',
       'field_39', 'field_40', 'field_41', 'field_42', 'field_43', 'field_44',
       'field_45', 'field_46', 'field_47', 'field_48', 'field_49', 'field_50',
       'field_51', 'field_52', 'field_53', 'field_54', 'field_55', 'field_56',
       'field_57', 'field_58', 'field_59', 'field_60', 'field_61', 'field_62',
       'field_63', 'field_64', 'field_65', 'field_66', 'field_67', 'code',
       'geometry'],
      dtype='object')

In [24]:
latest_routes_gdf.rename(columns={'code':'Código'},inplace=True)
latest_routes_gdf['layer']='' 
latest_routes_gdf['path']=''

latest_routes_gdf

,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_11,...,field_62,field_63,field_64,field_65,field_66,field_67,Código,geometry,layer,path
0,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001,"LINESTRING (668461.981 2288949.643, 668280.506...",,
1,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002,"LINESTRING (668277.775 2289017.683, 668281.117...",,
2,532,5/9/25 12:57:29,5/9/25 14:43:21,anonymous,NaN,NaN,Jesús Corona,5/24/2025,JC-1 V-004,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V004,"LINESTRING (668278.558 2289010.058, 668268.271...",,
3,533,5/9/25 14:43:23,5/9/25 14:50:40,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-1 V-005,TA_Providencia,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V005,"LINESTRING (668452.456 2288907.913, 668444.205...",,
4,536,5/9/25 15:08:24,5/9/25 15:20:28,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-1 M-008,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,JC1V008,"LINESTRING (667358.311 2288494.11, 667701.72 2...",,
5,546,5/11/25 20:07:30,5/11/25 20:15:38,anonymous,NaN,NaN,MeV,3/18/2025,003,TA_Providencia,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V003,"LINESTRING (668231.412 2288895.013, 668237.041...",,
6,548,5/11/25 20:18:06,5/11/25 20:29:49,anonymous,NaN,NaN,MeV,3/18/2025,005,TA_Providencia,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,MeV1V005,"LINESTRING (668278.155 2289010.77, 668283.05 2...",,
7,549,5/11/25 20:29:58,5/11/25 20:32:13,anonymous,NaN,NaN,MeV,3/18/2025,006,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V006,"LINESTRING (668189.196 2288623.242, 668205.104...",,
8,550,5/11/25 20:32:22,5/11/25 20:36:13,anonymous,NaN,NaN,MeV,3/18/2025,007,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,MeV1V007,"LINESTRING (668289.596 2289005.263, 668266.836...",,
9,551,5/11/25 20:36:23,5/11/25 20:41:02,anonymous,NaN,NaN,MeV,3/18/2025,008,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,MeV1V008,"LINESTRING (668266.836 2288945.916, 668240.895...",,


In [25]:
preliminar_routes = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/preliminar_routes.shp")
preliminar_routes.drop(columns=['fid','Codigo_2'],inplace=True)

# Show
print(preliminar_routes.crs)
print(preliminar_routes.shape)
preliminar_routes.head(2)

EPSG:32613
(184, 70)


,Código,layer,path,field_1,field_2,field_3,field_4,field_5,field_6,field_7,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,None,DP,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,None,DP,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."


In [26]:
final_routes = pd.concat([preliminar_routes,latest_routes_gdf])

# Show
print(final_routes.crs)
print(final_routes.shape)
final_routes.head(2)

EPSG:32613
(201, 70)


,Código,layer,path,field_1,field_2,field_3,field_4,field_5,field_6,field_7,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,None,DP,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,None,DP,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."


In [27]:
final_routes.tail(2)

,Código,layer,path,field_1,field_2,field_3,field_4,field_5,field_6,field_7,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
15,RV1V009,,,519,5/7/25 23:42:16,5/7/25 23:46:28,anonymous,NaN,NaN,RV,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,"LINESTRING (668394.401 2288804.953, 668040.401..."
16,RV1V010,,,520,5/7/25 23:46:32,5/7/25 23:50:05,anonymous,NaN,NaN,RV,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,En desacuerdo,De acuerdo,"LINESTRING (668168.761 2288845.455, 668226.395..."


## Export final data

In [29]:
final_routes.to_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/final_routes.gpkg")